In [12]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [13]:
data_dir = '/Users/saikarthik/Downloads/untitled folder/parkinsons_dataset'


In [ ]:
img_height, img_width = 224, 224 
batch_size = 32


In [15]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


In [16]:

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
)

Found 665 images belonging to 2 classes.


In [17]:

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
)

Found 166 images belonging to 2 classes.


In [18]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


In [19]:
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [20]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
)

Epoch 1/100


2025-04-02 11:07:33.151018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - ETA: 0s - loss: 0.6394 - accuracy: 0.6872

2025-04-02 11:07:38.620432: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 7s 302ms/step - loss: 0.6394 - accuracy: 0.6872 - val_loss: 0.6026 - val_accuracy: 0.7349
Epoch 2/100
21/21 [==============================] - 6s 259ms/step - loss: 0.6033 - accuracy: 0.7278 - val_loss: 0.5890 - val_accuracy: 0.7349
Epoch 3/100
21/21 [==============================] - 6s 264ms/step - loss: 0.5868 - accuracy: 0.7323 - val_loss: 0.5698 - val_accuracy: 0.7349
Epoch 4/100
21/21 [==============================] - 6s 266ms/step - loss: 0.5662 - accuracy: 0.7278 - val_loss: 0.5639 - val_accuracy: 0.7349
Epoch 5/100
21/21 [==============================] - 6s 263ms/step - loss: 0.5643 - accuracy: 0.7368 - val_loss: 0.5516 - val_accuracy: 0.7349
Epoch 6/100
21/21 [==============================] - 6s 262ms/step - loss: 0.5356 - accuracy: 0.7474 - val_loss: 0.5361 - val_accuracy: 0.7349
Epoch 7/100
21/21 [==============================] - 6s 270ms/step - loss: 0.5145 - accuracy: 0.7504 - val_loss: 0.5283 - val_accuracy: 0.7349
Epoch 8/100

In [25]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

validation_generator.reset()
predictions = (model.predict(validation_generator) > 0.5).astype("int64")
true_classes = validation_generator.classes

print("\nClassification Report:")
print(classification_report(true_classes, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, predictions))

6/6 [==============================] - 1s 166ms/step - loss: 0.3174 - accuracy: 0.8675
Validation Accuracy: 86.75%
6/6 [==============================] - 1s 195ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.85      0.79       122
           1       0.28      0.16      0.20        44

    accuracy                           0.67       166
   macro avg       0.51      0.51      0.50       166
weighted avg       0.62      0.67      0.64       166


Confusion Matrix:
[[104  18]
 [ 37   7]]
